# 第7章 常用的Keras功能

**列表7.1  Dropout層的使用範例**

In [1]:
# 內含Dropout層的Sequential模型範例
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout


model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))  # 將Dropout的rate設定為0.5
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


**列表7.2  BatchNormalization層的使用範例**

In [2]:
# 內含BatchNormalization層的Sequential模型範例
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, BatchNormalization


model = Sequential()
model.add(Dense(64, input_dim=20))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

**列表7.3  Lambda層的使用範例**

In [3]:
# 使用Lambda層的範例
from tensorflow.python.keras.layers import Input, Lambda
from tensorflow.python.keras.models import Model


model_in = Input(shape=(20,))
x = Lambda(lambda x: x/255.)(model_in)
x = Dense(64, activation='relu')(x)
model_out = Dense(10, activation='softmax')(x)

model = Model(inputs=model_in, outputs=model_out)
model.compile(
    loss='categorical_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

**列表7.4  利用層的參數新增活化函數的範例**

In [4]:
# 將relu或sigmoid指定為Dense層的參數，新增活化函數
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                1344      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________


**列表7.5  呼叫Activation層，建置與新增活化函數的範例**

In [6]:
# 呼叫Activation層，新增活化層
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.activations import relu


model = Sequential()
model.add(Dense(64, input_dim=20))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 64)                1344      
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________


**列表7.6  Clipped ReLU的範例**

In [8]:
# Clipped ReLU的範例
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.activations import relu


model = Sequential()
model.add(
    Dense(
        64, 
        input_dim=20, 
        activation=lambda x: relu(x, max_value=2)
    )
)
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='SGD',
    metrics=['accuracy']
)

**列表7.7  建置產生器**

In [9]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


# 建置ImageDataGenerator
# 指定具代表性的選項的範例
gen = ImageDataGenerator(
    rescale=1/255.,  # 尺度轉換 
    rotation_range=90.,  # 資料擴充相關選項 
    width_shift_range=1.,
    height_shift_range=.5,
    shear_range=.8,
    zoom_range=.5,
    horizontal_flip=True,
    vertical_flip=True
)

**列表7.8  產生迭代器與取得資料**

In [10]:
# 從資料夾載入影像與建置迭代器
iters = gen.flow_from_directory(
    'img',
    target_size=(32, 32),
    class_mode='binary',
    batch_size=5,
    shuffle=True
)

# 從迭代器取得1筆小批次的資料
x_train_batch, y_train_batch = next(iters)

print('shape of x_train_batch:', x_train_batch.shape)
print('shape of y_train_batch:', y_train_batch.shape)

Found 25 images belonging to 2 classes.
shape of x_train_batch: (5, 32, 32, 3)
shape of y_train_batch: (5,)


**列表7.9  利用迭代器學習模型**

In [11]:
import math
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Conv2D


# 建置分類專用的模型
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(32, 32, 3)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop'
)


# 計算學習幾筆小批次的資料，才算1個時期
steps_per_epoch = math.ceil(iters.samples/5)

# 將迭代器傳遞給`fit_generator`方法
histroy = model.fit_generator(
    iters, 
    steps_per_epoch=steps_per_epoch
)

Epoch 1/1
5/5 [==============================]5/5 [==============================] - 2s 338ms/step - loss: 2.1543

